In [617]:
import numpy as np
import pandas as pd
import datetime as dt

In [618]:
CURR_SOFR_FIX = 0.0455
FIX_DATE = dt.date(2023, 3, 2)

fomc_dates = [
    dt.date(2023, 3, 23),
    dt.date(2023, 5, 4),
    dt.date(2023, 6, 15),
    dt.date(2023, 7, 27),
    dt.date(2023, 9, 21),
    dt.date(2023, 12, 14)
    ]

start_steps = [25] * len(fomc_dates)

class SOFR1:
    pass


In [619]:

class CurveSOFR:

    def __init__(self, sofr_fixings: pd.DataFrame):
        
        self.fixings = sofr_fixings.copy()
        self.current_date = self.fixings.index[-1]

    def bootstrap_from_futures(self, fomc_dates: list, future_obj_list: list[SOFR1]):
        
        for future in future_obj_list:
            self.bootstrap_single_rate(fomc_dates, future)
            print(f'Bootstrapping rates for SOFR 1M future ending on {future.contract_end}')

    def bootstrap_single_rate(self, fomc_dates: list[dt.date], future_obj: SOFR1):
        
        start_date = future_obj.contract_start
        end_date = future_obj.contract_end
        next_fomc = min(fomc_dates, key=lambda sub: (sub - start_date) < dt.timedelta(0))
        
        # Check whether next FOMC meeting falls within contract month, if not, skip
        if next_fomc > end_date:
            return None

        days_in_contract = (end_date - start_date).days

        if (self.current_date < start_date):
            days_to_fomc = (next_fomc - start_date).days
        else:
            days_to_fomc = (next_fomc - self.current_date).days

        days_from_fomc = (end_date - next_fomc).days

        if start_date in self.fixings.index:
            fix_idx = start_date
        else:
            fix_idx = self.fixings.index[-1]

        observed = self.fixings[fix_idx:]

        r_boot = future_obj.bootstrap_rate_from_px(observed, days_in_contract, days_to_fomc, days_from_fomc)

        self.fixings.loc[next_fomc, 'rate'] = r_boot

class FuturesSOFR1M:

    def __init__(self, contract_start: dt.date, price = None):

        self.contract_start = contract_start
        self.contract_end = dt.date(
            self.contract_start.year + contract_start.month // 12, 
            contract_start.month % 12 + 1, 1
            ) - dt.timedelta(1)
        self.eval_date = None
        self.price = price
        self.implied_rate = (100 - self.price) / 100

    def price_from_curve(self, curve: CurveSOFR):
        
        pass

    def bootstrap_rate_from_px(self, observed: pd.DataFrame, days_in_contract: int, days_to_fomc: int, days_from_fomc: int):

        r_obs = np.prod(observed.rate) / len(observed)
        r_boot = (self.implied_rate * days_in_contract - r_obs * days_to_fomc) / days_from_fomc

        return r_boot

In [620]:
def load_fixings():
    sofr_fixings = pd.read_csv("sofr_fixings.csv").iloc[:-1,[0,2]]
    sofr_fixings.columns = ['date','rate']
    sofr_fixings.rate = sofr_fixings.rate / 100
    sofr_fixings.date = sofr_fixings.date.apply(lambda x: dt.datetime.strptime(x,"%m/%d/%Y").date())
    sofr_fixings.set_index('date', inplace=True)
    sofr_fixings = sofr_fixings.sort_index()
    return sofr_fixings

sofr_fixings = load_fixings()

In [621]:
SOFR = CurveSOFR(sofr_fixings)
future1 = FuturesSOFR1M(dt.date(2023, 3, 1), 95.365)
future2 = FuturesSOFR1M(dt.date(2023, 4, 1), 95.365)
future3 = FuturesSOFR1M(dt.date(2023, 5, 1), 95.3)
future4 = FuturesSOFR1M(dt.date(2023, 6, 1), 95.7)

futures_list = [future1, future2, future3, future4]

In [622]:
SOFR.bootstrap_from_futures(fomc_dates, futures_list)
SOFR.fixings 

Bootstrapping rates for SOFR 1M future ending on 2023-03-31
Bootstrapping rates for SOFR 1M future ending on 2023-04-30
Bootstrapping rates for SOFR 1M future ending on 2023-05-31
Bootstrapping rates for SOFR 1M future ending on 2023-06-30


,rate
date,
2023-01-25,0.043100
2023-01-26,0.043000
2023-01-27,0.043000
2023-01-30,0.043000
2023-01-31,0.043100
2023-02-01,0.043100
2023-02-02,0.045600
2023-02-03,0.045500
2023-02-06,0.045500
